In [1]:
N_EPOCHS = 50
batch_size = 32

# Flair

## Setup

In [2]:
import os, random, time
import numpy as np
import nibabel as nib
import cv2
import matplotlib.pyplot as plt

# Pytorch functions
import torch
# Neural network layers
import torch.nn as nn
import torch.nn.functional as F
# Optimizer
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset
# Torchvision library
from torchvision import transforms

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
# For results
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from torchsummary import summary

from utilities import *

In [3]:
# Device configuration
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
   device = torch.device('mps')
else:
    device = torch.device('cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

mps


In [4]:
def set_seed(seed, use_cuda = True, use_mps = False):
    """
    Set SEED for PyTorch reproducibility
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if use_cuda:
        torch.cuda.manual_seed_all(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    if use_mps:
        torch.mps.manual_seed(seed)

SEED = 44

USE_SEED = True

if USE_SEED:
    set_seed(SEED, torch.cuda.is_available(), torch.backends.mps.is_available())

## Define Custom Dataset

In [5]:
class BraTSDataset_Flair(Dataset):
    def __init__(self, image_path = r'./BraTS/BraTS2021_Training_Data_2D', transform=None):
        'Initialisation'
        self.image_path = image_path
        self.folders_name = [folder for folder in os.listdir(self.image_path) if folder != '.DS_Store']
        self.transform = transform

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.folders_name) * 155

    def __getitem__(self, index):
        'Generates one sample of data'

        # Determine the image index and the RGB layer
        image_idx = index // 155
        layer_idx = index % 155

        # Select sample
        file_name = self.folders_name[image_idx]
        
        path_img = os.path.join(self.image_path, file_name, 'flair', file_name + '_flair_' + str(layer_idx+1) + '.npy')
        image = np.load(path_img).astype(np.float32)

        path_label = os.path.join(self.image_path, file_name, 'seg', file_name + '_seg_' + str(layer_idx+1) + '.npy')
        label = np.load(path_label)
        
        if self.transform:
            image, label = self.transform([image, label])
        return image, label

In [6]:
class BinariseLabel(object):
    def __call__(self, sample):
        image, label = sample
        new_label = np.sign(label)
        return image, new_label

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    def __call__(self, sample):
        image, label = sample

        # numpy image: H x W x C
        # torch image: C x H x W
        # image = image.transpose((2, 0, 1))
        return torch.from_numpy(image), torch.from_numpy(label)

In [7]:
dataset = BraTSDataset_Flair(image_path = r'./BraTS/BraTS2021_Training_Data_2D',
                                    transform=transforms.Compose([
                                        BinariseLabel(),
                                        ToTensor()
                                    ]))

## Train Test Split

In [8]:
dataset_size = int(len(dataset)/155)
dataset_indices = list(range(dataset_size))

train_indices, test_indices = train_test_split(dataset_indices, test_size=0.1, random_state=SEED)
train_indices, val_indices = train_test_split(train_indices, test_size=0.22, random_state=SEED)

tmp_list = [[],[],[]]
for i, ind_list in enumerate([train_indices, val_indices, test_indices]):
    for ind in ind_list:
        for j in range(155):
            tmp_list[i].append(ind*155 + j)
train_indices, val_indices, test_indices = tmp_list

train_subset_Flair = Subset(dataset, train_indices)
val_subset_Flair = Subset(dataset, val_indices)
test_subset_Flair = Subset(dataset, test_indices)

# Create the subset DataLoader
train_dataloader_Flair = DataLoader(train_subset_Flair, batch_size=batch_size, shuffle=True)
val_dataloader_Flair = DataLoader(val_subset_Flair, batch_size=batch_size, shuffle=True)
test_dataloader_Flair = DataLoader(test_subset_Flair, batch_size=batch_size, shuffle=True)
# multiprocessing_context="forkserver", persistent_workers=True, num_workers = 4

## Define Convlution Autoencoder Structure

In [9]:
class ConvAutoencoder(nn.Module):
  def __init__(self):
    super().__init__()

    self.features = nn.Sequential(
      ## encoder layers ##
      # conv layer (depth from 1 --> 4), 3x3 kernels
      # Input 64 x 64
      nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, padding = 'same'), # 64 x 64
      nn.ReLU(),
      # pooling layer to reduce x-y dims by two; kernel and stride of 2
      nn.MaxPool2d(2), ## 32 x 32
      # conv layer (depth from 4 --> 8), 4x4 kernels
      nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, padding = 'same'), # 32 x 32
      nn.ReLU(),
      nn.MaxPool2d(2), # 16 x 16
      # conv layer (depth from 8 --> 12), 5x5 kernels
      nn.Conv2d(in_channels=8, out_channels=12, kernel_size=3, padding = 'same'), # ( 12 x ) 16 x 16
      nn.ReLU(),
      
      ## decoder layers ##
      # add transpose conv layers, with relu activation function
      nn.ConvTranspose2d(12, 6, kernel_size = 2, stride=2), # 32 x 32
      nn.ReLU(),
      nn.ConvTranspose2d(6, 1, kernel_size = 2, stride=2), # 64 x 64
      # output layer (with sigmoid for scaling from 0 to 1)
      # nn.Sigmoid()
    )
    
  def forward(self, x):
    x = x.view(int(np.prod(x.shape)/(64**2)), 1, 64, 64)
    x = self.features(x)
    # x = x.view(x.shape[0], -1)
    # x = torch.flatten(x, start_dim=1)
    return x

In [10]:
model = ConvAutoencoder().to(device)

print(f"The model has {count_parameters(model):,} trainable parameters.")

The model has 1,531 trainable parameters.


## Loss function and Optimisation Methods

In [11]:
# Loss
# criterion = nn.CrossEntropyLoss() # Softmax + CrossEntropy
# criterion = nn.BCELoss()
criterion = torch.nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

# Optim
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

## Model Training

In [12]:
train_losses_Flair, train_accs_Flair, valid_losses_Flair, valid_accs_Flair = model_training(N_EPOCHS,
                                                                    model,
                                                                    train_dataloader_Flair,
                                                                    val_dataloader_Flair,
                                                                    optimizer,
                                                                    criterion,
                                                                    device,
                                                                    './models/CA_Flair.pt')

100.0 % loaded in this epoch for evaluation.
Epoch: 1/50 -- Epoch Time: 171.56 s
---------------------------------
Train -- Loss: 0.696, Acc: 29.58%
Val -- Loss: 0.693, Acc: 26.39%
100.0 % loaded in this epoch for evaluation.
Epoch: 2/50 -- Epoch Time: 179.73 s
---------------------------------
Train -- Loss: 0.693, Acc: 61.51%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 3/50 -- Epoch Time: 178.59 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 4/50 -- Epoch Time: 180.66 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 5/50 -- Epoch Time: 179.01 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 6/50 -- Epoch Time: 178.04 s
--------------

# Repeat for T1

In [13]:
class BraTSDataset_T1(Dataset):
    def __init__(self, image_path = r'./BraTS/BraTS2021_Training_Data_2D', transform=None):
        'Initialisation'
        self.image_path = image_path
        self.folders_name = [folder for folder in os.listdir(self.image_path) if folder != '.DS_Store']
        self.transform = transform

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.folders_name) * 155

    def __getitem__(self, index):
        'Generates one sample of data'

        # Determine the image index and the RGB layer
        image_idx = index // 155
        layer_idx = index % 155

        # Select sample
        file_name = self.folders_name[image_idx]
        
        path_img = os.path.join(self.image_path, file_name, 't1', file_name + '_t1_' + str(layer_idx+1) + '.npy')
        image = np.load(path_img).astype(np.float32)

        path_label = os.path.join(self.image_path, file_name, 'seg', file_name + '_seg_' + str(layer_idx+1) + '.npy')
        label = np.load(path_label)
        
        if self.transform:
            image, label = self.transform([image, label])
        return image, label

In [14]:
dataset_T1 = BraTSDataset_T1(image_path = r'./BraTS/BraTS2021_Training_Data_2D',
                                    transform=transforms.Compose([
                                        BinariseLabel(),
                                        ToTensor()
                                    ]))

In [15]:
train_subset_T1 = Subset(dataset_T1, train_indices)
val_subset_T1 = Subset(dataset_T1, val_indices)
test_subset_T1 = Subset(dataset_T1, test_indices)

# Create the subset DataLoader
train_dataloader_T1 = DataLoader(train_subset_T1, batch_size=batch_size, shuffle=True)
val_dataloader_T1 = DataLoader(val_subset_T1, batch_size=batch_size, shuffle=True)
test_dataloader_T1 = DataLoader(test_subset_T1, batch_size=batch_size, shuffle=True)

In [16]:
# Reset Model Parameters

for layer in model.children():
   if hasattr(layer, 'reset_parameters'):
       layer.reset_parameters()

optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [17]:
train_losses_T1, train_accs_T1, valid_losses_T1, valid_accs_T1 = model_training(N_EPOCHS,
                                                                    model,
                                                                    train_dataloader_T1,
                                                                    val_dataloader_T1,
                                                                    optimizer,
                                                                    criterion,
                                                                    device,
                                                                    './models/CA_T1.pt')

100.0 % loaded in this epoch for evaluation.
Epoch: 1/50 -- Epoch Time: 213.21 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 2/50 -- Epoch Time: 207.02 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 3/50 -- Epoch Time: 208.49 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 4/50 -- Epoch Time: 207.48 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 5/50 -- Epoch Time: 208.15 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 6/50 -- Epoch Time: 207.52 s
--------------

# T1CE

In [18]:
class BraTSDataset_T1CE(Dataset):
    def __init__(self, image_path = r'./BraTS/BraTS2021_Training_Data_2D', transform=None):
        'Initialisation'
        self.image_path = image_path
        self.folders_name = [folder for folder in os.listdir(self.image_path) if folder != '.DS_Store']
        self.transform = transform

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.folders_name) * 155

    def __getitem__(self, index):
        'Generates one sample of data'

        # Determine the image index and the RGB layer
        image_idx = index // 155
        layer_idx = index % 155

        # Select sample
        file_name = self.folders_name[image_idx]
        
        path_img = os.path.join(self.image_path, file_name, 't1', file_name + '_t1_' + str(layer_idx+1) + '.npy')
        image = np.load(path_img).astype(np.float32)

        path_label = os.path.join(self.image_path, file_name, 'seg', file_name + '_seg_' + str(layer_idx+1) + '.npy')
        label = np.load(path_label)
        
        if self.transform:
            image, label = self.transform([image, label])
        return image, label

In [19]:
dataset_T1CE = BraTSDataset_T1CE(image_path = r'./BraTS/BraTS2021_Training_Data_2D',
                                    transform=transforms.Compose([
                                        BinariseLabel(),
                                        ToTensor()
                                    ]))

In [20]:
train_subset_T1CE = Subset(dataset_T1CE, train_indices)
val_subset_T1CE = Subset(dataset_T1CE, val_indices)
test_subset_T1CE = Subset(dataset_T1CE, test_indices)

# Create the subset DataLoader
train_dataloader_T1CE = DataLoader(train_subset_T1CE, batch_size=batch_size, shuffle=True)
val_dataloader_T1CE = DataLoader(val_subset_T1CE, batch_size=batch_size, shuffle=True)
test_dataloader_T1CE = DataLoader(test_subset_T1CE, batch_size=batch_size, shuffle=True)

In [21]:
# Reset Model Parameters

for layer in model.children():
   if hasattr(layer, 'reset_parameters'):
       layer.reset_parameters()

optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [22]:
train_losses_T1CE, train_accs_T1CE, valid_losses_T1CE, valid_accs_T1CE = model_training(N_EPOCHS,
                                                                    model,
                                                                    train_dataloader_T1CE,
                                                                    val_dataloader_T1CE,
                                                                    optimizer,
                                                                    criterion,
                                                                    device,
                                                                    './models/CA_T1CE.pt')

100.0 % loaded in this epoch for evaluation.
Epoch: 1/50 -- Epoch Time: 207.35 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 2/50 -- Epoch Time: 203.97 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 3/50 -- Epoch Time: 204.87 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 4/50 -- Epoch Time: 204.01 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 5/50 -- Epoch Time: 204.13 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 6/50 -- Epoch Time: 206.05 s
--------------

# T2

In [23]:
class BraTSDataset_T2(Dataset):
    def __init__(self, image_path = r'./BraTS/BraTS2021_Training_Data_2D', transform=None):
        'Initialisation'
        self.image_path = image_path
        self.folders_name = [folder for folder in os.listdir(self.image_path) if folder != '.DS_Store']
        self.transform = transform

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.folders_name) * 155

    def __getitem__(self, index):
        'Generates one sample of data'

        # Determine the image index and the RGB layer
        image_idx = index // 155
        layer_idx = index % 155

        # Select sample
        file_name = self.folders_name[image_idx]
        
        path_img = os.path.join(self.image_path, file_name, 't1', file_name + '_t1_' + str(layer_idx+1) + '.npy')
        image = np.load(path_img).astype(np.float32)

        path_label = os.path.join(self.image_path, file_name, 'seg', file_name + '_seg_' + str(layer_idx+1) + '.npy')
        label = np.load(path_label)
        
        if self.transform:
            image, label = self.transform([image, label])
        return image, label

In [24]:
dataset_T2 = BraTSDataset_T2(image_path = r'./BraTS/BraTS2021_Training_Data_2D',
                                    transform=transforms.Compose([
                                        BinariseLabel(),
                                        ToTensor()
                                    ]))

In [25]:
train_subset_T2 = Subset(dataset_T2, train_indices)
val_subset_T2 = Subset(dataset_T2, val_indices)
test_subset_T2 = Subset(dataset_T2, test_indices)

# Create the subset DataLoader
train_dataloader_T2 = DataLoader(train_subset_T2, batch_size=batch_size, shuffle=True)
val_dataloader_T2 = DataLoader(val_subset_T2, batch_size=batch_size, shuffle=True)
test_dataloader_T2 = DataLoader(test_subset_T2, batch_size=batch_size, shuffle=True)

In [26]:
# Reset Model Parameters

for layer in model.children():
   if hasattr(layer, 'reset_parameters'):
       layer.reset_parameters()

optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [27]:
train_losses_T2, train_accs_T2, valid_losses_T2, valid_accs_T2 = model_training(N_EPOCHS,
                                                                    model,
                                                                    train_dataloader_T2,
                                                                    val_dataloader_T2,
                                                                    optimizer,
                                                                    criterion,
                                                                    device,
                                                                    './models/CA_T2.pt')

100.0 % loaded in this epoch for evaluation.
Epoch: 1/50 -- Epoch Time: 203.63 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 2/50 -- Epoch Time: 204.72 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 3/50 -- Epoch Time: 202.99 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 4/50 -- Epoch Time: 205.03 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 5/50 -- Epoch Time: 206.05 s
---------------------------------
Train -- Loss: 0.693, Acc: 97.33%
Val -- Loss: 0.693, Acc: 97.22%
100.0 % loaded in this epoch for evaluation.
Epoch: 6/50 -- Epoch Time: 204.00 s
--------------

## Save train, val and test indices, losses and accuracies

In [28]:
# train_losses, train_accs, valid_losses, valid_accs
# train_losses_T1, train_accs_T1, valid_losses_T1, valid_accs_T1
# train_losses_T1CE, train_accs_T1CE, valid_losses_T1CE, valid_accs_T1CE
# train_losses_T2, train_accs_T2, valid_losses_T2, valid_accs_T2